In [3]:
import meow as mw
import numpy as np
import matplotlib.pyplot as plt

## Preparation
let's perform a mode simulation

In [4]:
length = 10.0
box = mw.Box(
    x_min=-0.22,
    x_max=0.22,
    y_min=0,
    y_max=0.22,
    z_min=0.0,
    z_max=length,
)

In [5]:
struct = mw.Structure(material=mw.silicon, geometry=box)

In [6]:
env = mw.Environment(wl=1.55, T=25.0)

In [7]:
cells = mw.create_cells(
    structures=[struct],
    mesh=mw.Mesh2d(
        x = np.linspace(-1, 1, 101),
        y = np.linspace(-1, 1, 101),
    ),
    Ls = [length/3]*3
)
css = [mw.CrossSection(cell=cell, env=env) for cell in cells]

In [8]:
modes = [mw.compute_modes(cs, num_modes=2) for cs in css]

In [33]:
from meow.eme import compute_interface_s_matrices, compute_propagation_s_matrices
from meow.eme import compute_interface_s_matrix
from meow.eme.sax import _get_netlist
import sax
from sax.backends import circuit_backends
evaluate_circuit = circuit_backends["klu"]

In [38]:
propagations = compute_propagation_s_matrices(modes)
interfaces = compute_interface_s_matrices(
    modes, enforce_reciprocity=False,
)

unities = [compute_interface_s_matrix(mode, mode, enforce_reciprocity=False) for mode in modes]


## TODO get rid of unities to increase efficiency

In [73]:
def _connect_two(l, r):
    """l -> left, r -> right"""
    #TODO there must be an easier way to do this...
    s_l, p_l = sax.sdense(l)
    s_r, p_r = sax.sdense(r)
    instances = {"l": l, "r": r}
    p_lr = [p for p in p_l.keys() if "right" in p] #right ports of left
    p_rl = [p for p in p_r.keys() if "left" in p] #left ports of right

    p_ll = [p for p in p_l.keys() if "left" in p] #left ports of left
    p_rr = [p for p in p_r.keys() if "right" in p] #right ports of right

    p_lr.sort()
    p_rl.sort()
    connections = {f"l,{pl}": f"r,{pr}" for pl,pr in zip(p_lr, p_rl)}
    ports = {
        **{p: f"l,{p}" for p in p_ll},
        **{p: f"r,{p}" for p in p_rr}
    }
    net = dict(
        instances = instances,
        connections = connections,
        ports = ports
    )
    return evaluate_circuit(**net)

In [86]:
def pi_pairs(propagations, interfaces, unities):
    """generates the S-matrices of cells: a combination of propagation and interface matrix"""
    S = []
    for i in range(len(propagations)):
        p = propagations[f"p_{i}"]
        if i == len(interfaces):
            S.append(p)
        else:
            c = interfaces[f"i_{i}_{i+1}"]
            S.append(_connect_two(p, c))
    
    return S

In [87]:
pairs = pi_pairs(propagations, interfaces, unities)

In [88]:
def l2r_matrices(pairs):
    Ss=[pairs[0]]

    for p in pairs[1:]:
        Ss.append(_connect_two(Ss[-1], p))
    
    return Ss
    
l2r_matrices(pairs)

[(Array([[ 7.14851583e-22+4.92011923e-22j,  9.04807018e-11-3.31219244e-11j,
           9.54920193e-01+2.96862637e-01j, -8.60058235e-12-2.67372245e-12j],
         [-1.04431652e-10+3.82288953e-11j,  2.63447864e-22-9.66346452e-22j,
           7.15731899e-12-5.46728267e-12j,  7.94677102e-01-6.07032375e-01j],
         [ 9.54920193e-01+2.96862637e-01j, -7.15733724e-12+5.46729661e-12j,
           8.67806729e-22-1.76601327e-44j,  9.63525780e-11+1.17997879e-26j],
         [ 8.60056042e-12+2.67371564e-12j,  7.94677102e-01-6.07032375e-01j,
          -1.11208895e-10+1.36191614e-26j,  1.00161382e-21-8.11315872e-43j]],      dtype=complex128),
  {'left@0': 0, 'left@1': 1, 'right@0': 2, 'right@1': 3}),
 (Array([[ 2.83518070e-21+1.76853157e-21j,  1.64061458e-10-9.53287699e-11j,
           8.23745149e-01+5.66960254e-01j, -1.58768634e-11-2.01029720e-12j],
         [-1.89357606e-10+1.10027229e-10j, -7.57843898e-22-3.33739914e-21j,
           1.08266403e-11-1.17855267e-11j,  2.63023392e-01-9.64789456e-01j]

In [97]:
def r2l_matrices(pairs):
    Ss=[pairs[-1]]

    for p in pairs[-1::-1]:
        Ss.append(_connect_two(p, Ss[-1]))
    
    return Ss

r2l_matrices(pairs)

[{('left@0', 'right@0'): 9.549e-01+2.969e-01j,
  ('left@1', 'right@1'): 7.947e-01-6.070e-01j,
  ('right@0', 'left@0'): 9.549e-01+2.969e-01j,
  ('right@1', 'left@1'): 7.947e-01-6.070e-01j},
 (Array([[0.        +0.j        , 0.        +0.j        ,
          0.82374515+0.56696025j, 0.        +0.j        ],
         [0.        +0.j        , 0.        +0.j        ,
          0.        +0.j        , 0.26302339-0.96478946j],
         [0.82374515+0.56696025j, 0.        +0.j        ,
          0.        +0.j        , 0.        +0.j        ],
         [0.        +0.j        , 0.26302339-0.96478946j,
          0.        +0.j        , 0.        +0.j        ]],      dtype=complex128),
  {'left@0': 0, 'left@1': 1, 'right@0': 2, 'right@1': 3}),
 (Array([[ 7.14851583e-22+4.92011923e-22j,  9.04807018e-11-3.31219244e-11j,
           6.18301560e-01+7.85940952e-01j, -4.84173358e-12+7.59449962e-12j],
         [-1.04431652e-10+3.82288953e-11j,  2.63447864e-22-9.66346452e-22j,
           8.99553877e-12-4.45

In [91]:
l = list(range(5))

In [96]:
l[-1::-1]

[4, 3, 2, 1, 0]